In [2]:
import numpy as np
import pandas as pd
import re

In [3]:
data = pd.read_csv("SuperStore_Dataset.csv")

In [13]:
if data['Quantity'].isnull().sum() > 0:
    print("There are missing values in 'Quantity' column.")

There are missing values in 'Quantity' column.


Row ID        False
Order ID      False
Order Date    False
Ship Date     False
Ship Mode      True
dtype: bool

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10014 entries, 0 to 10013
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         10014 non-null  int64  
 1   Order ID       10014 non-null  object 
 2   Order Date     10014 non-null  object 
 3   Ship Date      10014 non-null  object 
 4   Ship Mode      9916 non-null   object 
 5   Customer ID    10014 non-null  object 
 6   Customer Name  10014 non-null  object 
 7   Segment        10014 non-null  object 
 8   Country        10014 non-null  object 
 9   City           10014 non-null  object 
 10  State          10014 non-null  object 
 11  Postal Code    10014 non-null  int64  
 12  Region         10014 non-null  object 
 13  Product ID     10014 non-null  object 
 14  Category       10014 non-null  object 
 15  Sub-Category   10014 non-null  object 
 16  Product Name   10014 non-null  object 
 17  Sales Price    10014 non-null  float64
 18  Quanti

In [6]:
data['Quantity'].head(40)

0     2.0
1     3.0
2     2.0
3     5.0
4     2.0
5     7.0
6     4.0
7     6.0
8     3.0
9     5.0
10    9.0
11    4.0
12    3.0
13    NaN
14    5.0
15    NaN
16    6.0
17    2.0
18    2.0
19    NaN
20    4.0
21    7.0
22    7.0
23    2.0
24    NaN
25    2.0
26    NaN
27    7.0
28    2.0
29    NaN
30    2.0
31    6.0
32    6.0
33    2.0
34    3.0
35    7.0
36    5.0
37    9.0
38    3.0
39    3.0
Name: Quantity, dtype: float64

In [7]:
import pandas as pd
import numpy as np
import re

def smart_convert_dates_and_numbers(df):
    """
    Automatically detects and converts columns to datetime, int, or float.
    Returns the converted DataFrame and a summary log DataFrame.
    """
    log_entries = []

    # Regex patterns
    date_regex = r'^\s*(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\d{4}[-/]\d{1,2}[-/]\d{1,2}|[A-Za-z]{3,9}\s+\d{1,2},?\s+\d{2,4})\s*$'
    numeric_regex = r'^[\+\-]?(?:\d+|\d{1,3}(?:,\d{3})+)?(?:\.\d+)?(?:[eE][\+\-]?\d+)?$'

    for col in df.columns:
        old_type = df[col].dtype
        series = df[col].astype(str).str.strip()
        non_null = series[series.notna() & (series != 'nan')]

        if non_null.empty:
            log_entries.append([col, old_type, old_type, 0, "empty/unchanged"])
            continue

        # --- 1️⃣ Detect & Convert Datetime FIRST ---
        date_mask = non_null.str.match(date_regex, na=False)
        date_ratio = date_mask.sum() / len(non_null)

        if date_ratio > 0.5 or any(k in col.lower() for k in ("date", "time", "ship", "order", "dob")):
            parsed = pd.to_datetime(df[col], errors='coerce', dayfirst=True, infer_datetime_format=True)
            success = parsed.notna().sum() / len(df[col])
            if success > 0.5:
                df[col] = parsed
                log_entries.append([col, old_type, "datetime64[ns]", round(success * 100, 2), "date pattern matched"])
                continue

        # --- 2️⃣ Detect & Convert Numeric ---
        candidate = non_null[~non_null.str.match(date_regex, na=False)]
        cand_clean = candidate.str.replace(r'\s+', '', regex=True).str.replace(',', '', regex=False)
        numeric_mask = cand_clean.str.match(numeric_regex, na=False)
        numeric_ratio = numeric_mask.sum() / len(non_null)

        if numeric_ratio > 0.8:
            cleaned = non_null.str.replace(r'[^\d\.\-\+eE,]', '', regex=True).str.replace(',', '', regex=False)
            df[col] = pd.to_numeric(cleaned, errors='coerce')

            if np.all(df[col].dropna() == np.floor(df[col].dropna())):
                df[col] = df[col].astype("Int64")
                new_type = "int"
            else:
                new_type = "float"

            log_entries.append([col, old_type, new_type, round(numeric_ratio * 100, 2), "numeric pattern matched"])
            continue

        # --- 3️⃣ Leave Unchanged ---
        log_entries.append([col, old_type, old_type, 0, "unchanged"])

    # --- 🧾 Build Conversion Log ---
    log_df = pd.DataFrame(log_entries, columns=["Column", "Old Type", "New Type", "Confidence (%)", "Reason"])
    return df, log_df


In [8]:
df = pd.read_csv("SuperStore_Dataset.csv")
converted_df, conversion_log = smart_convert_dates_and_numbers(df)

print(conversion_log)

C:\Users\auau\AppData\Local\Temp\ipykernel_29388\116200622.py:30: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(df[col], errors='coerce', dayfirst=True, infer_datetime_format=True)
C:\Users\auau\AppData\Local\Temp\ipykernel_29388\116200622.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[col], errors='coerce', dayfirst=True, infer_datetime_format=True)
C:\Users\auau\AppData\Local\Temp\ipykernel_29388\116200622.py:30: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://panda

           Column Old Type        New Type  Confidence (%)  \
0          Row ID    int64             int           100.0   
1        Order ID   object          object             0.0   
2      Order Date   object  datetime64[ns]           100.0   
3       Ship Date   object  datetime64[ns]           100.0   
4       Ship Mode   object          object             0.0   
5     Customer ID   object          object             0.0   
6   Customer Name   object          object             0.0   
7         Segment   object          object             0.0   
8         Country   object          object             0.0   
9            City   object          object             0.0   
10          State   object          object             0.0   
11    Postal Code    int64             int           100.0   
12         Region   object          object             0.0   
13     Product ID   object          object             0.0   
14       Category   object          object             0.0   
15   Sub

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10014 entries, 0 to 10013
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         10014 non-null  Int64         
 1   Order ID       10014 non-null  object        
 2   Order Date     10014 non-null  datetime64[ns]
 3   Ship Date      10014 non-null  datetime64[ns]
 4   Ship Mode      9916 non-null   object        
 5   Customer ID    10014 non-null  object        
 6   Customer Name  10014 non-null  object        
 7   Segment        10014 non-null  object        
 8   Country        10014 non-null  object        
 9   City           10014 non-null  object        
 10  State          10014 non-null  object        
 11  Postal Code    10014 non-null  Int64         
 12  Region         10014 non-null  object        
 13  Product ID     10014 non-null  object        
 14  Category       10014 non-null  object        
 15  Sub-Category   1001